<a href="https://colab.research.google.com/github/37stu37/AI/blob/master/ctgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**import**
---



In [0]:
import pandas as pd
import pathlib as Path

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/01_githubRepo/GNS_GAN

/content/drive/My Drive/Colab Notebooks/01_githubRepo/GNS_GAN


In [0]:
outputDir = '../../02_output'

In [0]:
data = pd.read_parquet(outputDir + '/smallMonteCarlo2500runs.parquet')

**Data processing**
---



In [0]:
data.reset_index(inplace=True)
data.drop(['index', 'scenario', 'pid'], axis=1, inplace=True)

In [6]:
len(data) #; data.columns

7864124

In [0]:
trainingSample = data.sample(n=70000)
discrete_columns = ['source', 'target']

In [8]:
trainingSample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 7738481 to 5877497
Data columns (total 5 columns):
source        70000 non-null int64
target        70000 non-null int64
distance      70000 non-null float64
bearing       70000 non-null float64
IgnProb_bl    70000 non-null float64
dtypes: float64(3), int64(2)
memory usage: 3.2 MB


**CTGAN**

In [0]:
%%capture
!pip install ctgan 

In [0]:
%%time
from ctgan import CTGANSynthesizer

ctgan = CTGANSynthesizer()
ctgan.fit(trainingSample, discrete_columns, epochs=75)

**Generate synthetic data**

In [11]:
%%time
samples = ctgan.sample(1000)

CPU times: user 56.1 ms, sys: 5.69 ms, total: 61.8 ms
Wall time: 78.5 ms


**Validation**
---



In [0]:
from FFEpostprocessing.ipynb import load_shapefile, count_fid_occurences, merge_coordinates_export_shape # comment out some def

In [0]:
shapefile = load_shapefile("buildings_raw.shp", 1748570, 5425500, 1749500, 5427600) # small

In [0]:
countGAN = count_fid_occurences(samples)
countGAN = countGAN[(countGAN.source.isin(shapefile.source)) & (countGAN.target.isin(shapefile.target))]
print(len(countGAN))
countGAN = merge_coordinates_export_shape(count_df, gdf, "smallBurnedBuildings")